In [98]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [99]:
import os
import sqlite3
import pandas as pd
import requests
API_KEY = str(os.getenv("LASTFM_KEY"))
API_SECRET = str(os.getenv("LASTFM_SECRET"))
headers = {
    'user-agent': "S&DS 230"
}

In [104]:
from tqdm.auto import tqdm
tqdm.pandas()

/Users/mattshu/.pyenv/versions/3.8.3/lib/python3.8/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [100]:
SQL_FILEPATH = os.getenv("PLAYBACK_FILE")
lyrics_db = sqlite3.connect(SQL_FILEPATH)
df = pd.read_sql_query("SELECT * FROM tracks", lyrics_db)

In [101]:
def find_track(artist: str, track: str):
    payload = {
        'api_key': API_KEY,
        'method': 'track.search',
        'format': 'json',
        'track': track,
        'artist': artist,
    }
    try:
        r = requests.get('https://ws.audioscrobbler.com/2.0/', headers=headers, params=payload)
        if r.status_code != 200:
            return None
        numResults = int(r.json()["results"]["opensearch:totalResults"])
        if numResults > 0:
            matches = r.json()["results"]["trackmatches"]["track"][0]["url"]
            listeners = r.json()["results"]["trackmatches"]["track"][0]["listeners"]
            return {"last_fm_url": matches, "last_fm_listeners": listeners}
        else:
            return None
    except requests.exceptions.RequestException as e:  # This is the correct syntax
        return None

In [102]:
def find_track_series(series):
    track_data = find_track(series["artist"], series["track"])
    if track_data:
        series["last_fm_url"] = track_data["last_fm_url"]
        series["last_fm_listeners"] = track_data["last_fm_listeners"]
    else:
        series["last_fm_url"] = None
        series["last_fm_listeners"] = None
    return series

In [105]:
df = df.progress_apply(find_track_series, axis=1).drop("level_0", axis=1).drop("index", axis=1)

ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html

In [ ]:
df.to_sql("tracks", lyrics_db, if_exists="replace")